In [91]:
library(reticulate)
library(fitHeavyTail)
library(jsonlite)
library(abind)
# np <- import("numpy")

In [20]:
help(fit_mvt)

fit_mvt {fitHeavyTail},R Documentation
X,Data matrix containing the multivariate time series (each column is one time series).
na_rm,"Logical value indicating whether to remove observations with some NAs (default is TRUE). Otherwise, the NAs will be imputed at a higher computational cost."
nu,"Degrees of freedom of the t distribution. Either a number (>2) or a string indicating the method to compute it: ""iterative"": iterative estimation (with method to be specified in argument nu_iterative_method) with the rest of the parameters (default method); ""kurtosis"": one-shot estimation based on the kurtosis obtained from the sampled moments; ""MLE-diag"": one-shot estimation based on the MLE assuming a diagonal sample covariance; ""MLE-diag-resampled"": like method ""MLE-diag"" but resampled for better stability."
nu_iterative_method,"String indicating the method for iteratively estimating nu (in case nu = ""iterative""): ""ECM"": maximization of the Q function [Liu-Rubin, 1995]; ""ECME"": maximization of the log-likelihood function [Liu-Rubin, 1995]; ""OPP"": estimator from paper [Ollila-Palomar-Pascal, TSP2021, Alg. 1]; ""OPP-harmonic"": variation of ""OPP""; ""POP"": improved estimator as in paper [Pascal-Ollila-Palomar, EUSIPCO2021, Alg. 1] (default method)."
initial,"List of initial values of the parameters for the iterative estimation method (in case nu = ""iterative""). Possible elements include: mu: default is the data sample mean, cov: default is the data sample covariance matrix, scatter: default follows from the scaled sample covariance matrix, nu: can take the same values as argument nu, default is 4, B: default is the top eigenvectors of initial$cov multiplied by the sqrt of the eigenvalues, psi: default is diag(initial$cov - initial$B %*% t(initial$B))."
optimize_mu,Boolean indicating whether to optimize mu (default is TRUE).
weights,Optional weights for each of the observations (the length should be equal to the number of rows of X).
scale_covmat,Logical value indicating whether to scale the scatter and covariance matrices to minimize the MSE estimation error by introducing bias (default is FALSE). This is particularly advantageous when the number of observations is small compared to the number of variables.
PX_EM_acceleration,"Logical value indicating whether to accelerate the iterative method via the PX-EM acceleration technique (default is TRUE) [Liu-Rubin-Wu, 1998]."
nu_update_start_at_iter,"Starting iteration (default is 1) for iteratively estimating nu (in case nu = ""iterative"")."


In [4]:
samples_vgmcp <- fromJSON("secondary-data/samples_vgmcp_r.json", flatten=TRUE)
weights <- fromJSON("secondary-data/weights_r.json", flatten=TRUE)

In [193]:
convergence <- list()
nus = array(numeric(), c(length(samples_vgmcp))) 
means = array(numeric(), c(length(samples_vgmcp), 3)) 
covs = array(numeric(), c(length(samples_vgmcp), 3, 3))
for (i in seq(1, length(samples_vgmcp))) {
    est_mvt = fit_mvt(samples_vgmcp[[i]], weights=weights[[i]], nu_iterative_method='ECME', nu='MLE-diag')
    nus[i] <- est_mvt$nu
    means[i, 1:3] <- array(est_mvt$mean)
    covs[i, 1:3, 1:3] <- est_mvt$cov
    convergence <- append(convergence, est_mvt$converged)
}

In [195]:
# save all estimated parameters
reticulate::r_to_py(means)$dump('secondary-data/r_to_py_mvt_means.npy')
reticulate::r_to_py(covs)$dump('secondary-data/r_to_py_mvt_covs.npy')
reticulate::r_to_py(nus)$dump('secondary-data/r_to_py_mvt_dofs.npy')

None

None

None

In [194]:
means[1:5, 1:3]
covs[1, 1:3, 1:3]
nus
convergence

5.516572,-5.3941583,-4.3663873
-1.091628,0.4190871,4.7865871
1.822593,3.5872000,0.5886099
6.728251,-6.2492406,-4.6230636
-5.397431,3.2744442,4.8385694


3.18079904,0.2026469,0.09212282
0.20264692,2.2503964,-0.49515121
0.09212282,-0.4951512,1.36950687


[1] 99.999945 51.658342 20.981826 11.000811 99.999922 38.843333 99.999919
 [8] 99.999928 99.999936 99.999942  5.509039 98.220904 99.999930  6.173832
[15] 99.999921 62.656876  9.198198 33.084846 99.999924 10.154756 29.726504
[22] 99.999920 19.225767 19.052623 99.999917 10.758873 17.164233 99.999917
[29] 31.013826 99.999920 99.999919 35.942767 99.999916 99.999926 11.395368
[36] 16.204792 27.956126 99.999921 99.999923 99.999948 99.999931 99.999916
[43] 30.810088 99.999922 44.317517 99.999924 99.999922 99.999922 19.474775
[50] 43.985518 33.873453 99.999926 99.999922 99.999924 28.796273  9.869002
[57] 99.999932 99.999920 21.294794 80.784838 99.999921 91.809176 16.990173
[64] 99.999952 99.999940 29.630242 99.999922 12.032395 25.343524

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

[[10]]
[1] TRUE

[[11]]
[1] TRUE

[[12]]
[1] TRUE

[[13]]
[1] TRUE

[[14]]
[1] TRUE

[[15]]
[1] TRUE

[[16]]
[1] TRUE

[[17]]
[1] TRUE

[[18]]
[1] TRUE

[[19]]
[1] TRUE

[[20]]
[1] TRUE

[[21]]
[1] TRUE

[[22]]
[1] TRUE

[[23]]
[1] TRUE

[[24]]
[1] TRUE

[[25]]
[1] TRUE

[[26]]
[1] TRUE

[[27]]
[1] TRUE

[[28]]
[1] TRUE

[[29]]
[1] TRUE

[[30]]
[1] TRUE

[[31]]
[1] TRUE

[[32]]
[1] TRUE

[[33]]
[1] TRUE

[[34]]
[1] TRUE

[[35]]
[1] TRUE

[[36]]
[1] TRUE

[[37]]
[1] TRUE

[[38]]
[1] TRUE

[[39]]
[1] TRUE

[[40]]
[1] TRUE

[[41]]
[1] TRUE

[[42]]
[1] TRUE

[[43]]
[1] TRUE

[[44]]
[1] TRUE

[[45]]
[1] TRUE

[[46]]
[1] TRUE

[[47]]
[1] TRUE

[[48]]
[1] TRUE

[[49]]
[1] TRUE

[[50]]
[1] TRUE

[[51]]
[1] TRUE

[[52]]
[1] TRUE

[[53]]
[1] TRUE

[[54]]
[1] TRUE

[[55]]
[1] TRUE

[[56]]
[1] TRUE

[[57]]
[1] TRUE

[[58]]
[1] TRUE

[[59]]
[1] TRUE

[[60]]
[1] TRUE

[[61]]
[1] TRUE

[[62]]
[1] TRUE

[[63]]
[1] TRUE

[[64]]
[1] TRUE

[[65]]
[1] TRUE

[[66]]
[1] TRUE

[[67]]
[1] TRUE

[[68]]
[1] TRUE

[[69]]
[1] TRUE

In [172]:
est_mvt

,V1,V2,V3
V1,6.0650053,0.5194964,-2.0561802
V2,0.5194964,2.2439269,-0.9073054
V3,-2.0561802,-0.9073054,2.5994378
,V1,V2,V3
V1,6.1887810,0.5300983,-2.0981431
V2,0.5300983,2.2897213,-0.9258219
V3,-2.0981431,-0.9258219,2.6524876
